# Temporal view of Henry VIII's marriages

In [ ]:
# Data

wives = [
    {'name': "Catherine of Aragon", 'marriage_start_date': "1509-06-11", 'marriage_end_date': "1533-05-23"},
    # Henry had already secretly exchanged wedding vows with Anne on 1532-11-14 (before Cranmer ruled the 
    # marriage to Catherine null and void). However, in order to avoid an awkward overlap, we go with a 
    # marriage_start_date for Anne of 1533-05-28, which is when Cranmer pronounced the King legally 
    # married to Anne.
    {'name': "Anne Boleyn", 'marriage_start_date': "1533-05-28", 'marriage_end_date': "1536-05-17"},
    {'name': "Jane Seymour", 'marriage_start_date': "1536-05-30", 'marriage_end_date': "1537-10-24"},
    {'name': "Anne of Cleves", 'marriage_start_date': "1540-01-06", 'marriage_end_date': "1540-07-12"},
    {'name': "Catherine Howard", 'marriage_start_date': "1540-07-28", 'marriage_end_date': "1542-02-13"},
    {'name': "Catherine Parr", 'marriage_start_date': "1543-07-12", 'marriage_end_date': "1547-01-28"}
]

In [ ]:
# Process the data

from datetime import date

# To parse dates, e.g.: d = date.fromisoformat("1990-05-17")

# TODO: Sanity check that the marriage dates are consecutive and non-overlapping
